In [ ]:
import openai
import backoff
from bardapi import Bard
import csv
import random
import time


openai.api_key = "" 

@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_tries = 5)
def get_answer_from_openai(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result

def get_answer_from_gpt4(file, prompt):
    
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result


def get_answer_from_bard(prompt):
    bard_token_list =  []
    random.shuffle(bard_token_list)

    for bard_token in bard_token_list:
        try:
            for i in range(5):
                result = Bard(token=bard_token, timeout=30).get_answer(prompt)["content"]
                if ('Response Error:' not in result):
                    return result
        except:   
            print('no') 
            continue
#bard < GPT4

In [ ]:
#여기에서 사용하는 gpt4 사용코드
def get_answer_from_gpt4_test(prompt):
    
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result

In [ ]:
accommodation_type = '료칸'
hotel = ['영어 가능한 직원', '골프장', '주차장', '친절한 직원', '로맨틱', '큰 호텔', '자쿠지', '리노베이팅', '가족여행', '넓은 방', '스파', '관광 할인', '바', '무인 체크인', '아늑함', '에어컨', '레이트 체크아웃', '편안함', '인피니티 풀', '신혼여행', '치안 좋은', '통유리 창문', '통풍 잘되는', '버스 정류장 앞', '맛집', '키즈클럽', '가까운 해변', '쇼핑몰', '도심', '휴양지 느낌', '셔틀버스 운영', '얼리 체크인', '온천', '관광지', '24시간 프런트', '작은 호텔', '주차장', '숙박', '빠른 대응', '네임밸류', '위생적인', '도보 가능', '조식', '매일 청소', '세심함', '어메니티', '가성비', '공항에서 가까운', '바다 뷰', '가족여행']
resort = ['영어 가능한 직원', '골프장', '주차장', '친절한 직원', '로맨틱', '큰 호텔', '자쿠지', '리노베이팅', '가족여행', '넓은 방', '스파', '관광 할인', '바', '무인 체크인', '아늑함', '에어컨', '레이트 체크아웃', '편안함', '인피니티 풀', '신혼여행', '치안 좋은', '통유리 창문', '통풍 잘되는', '버스 정류장 앞', '맛집', '키즈클럽', '가까운 해변', '쇼핑몰', '도심', '휴양지 느낌', '셔틀버스 운영', '얼리 체크인', '온천', '관광지', '24시간 프런트', '작은 호텔', '주차장', '숙박', '빠른 대응', '네임밸류', '위생적인', '도보 가능', '조식', '매일 청소', '썬베드', '수상 레저', '세심함', '어메니티', '가성비', '공항에서 가까운', '바다 뷰', '가족여행']
ryokan = ['영어 가능한 직원', '주차장', '친절한 직원', '로맨틱','일본식', '히노끼 탕', '전통적인', '리노베이팅', '가족여행', '넓은 방', '스파', '관광 할인', '무인 체크인', '아늑함', '에어컨', '레이트 체크아웃', '편안함', '가이세키 요리', '목욕탕', '온천', '신혼여행', '치안 좋은', '다다미 방', '통풍 잘되는', '버스 정류장 앞', '맛집', '쇼핑몰', '도심', '셔틀버스 운영', '얼리 체크인', '온천', '관광지', '24시간 프런트', '주차장', '숙박', '위생적인', '도보 가능', '조식', '매일 청소', '세심함', '호텔식', '가성비', '공항에서 가까운', '바다 뷰', '가족여행']

숙소 바꿀 때에는:
1) 위 accommodation_type를 원하는 숙소 type으로 바꿔준다 (호텔, 리조트, 료칸, 등)
2) 아래 keywords를 accommodation_type에 상응하는 keywords list이름으로 바꿔준다
    (키워드가 숙소별로 다르게 설정되어 있기 때문)
3) 필요 시, 원하는 키워드 추가 / 제거로 더욱 focused한 output 생성 가능

In [ ]:
from tqdm.auto import tqdm
keywords = ryokan #바꾸기
#키워드는 호텔/리조트/료칸에 따라 바꿔서 사용

@backoff.on_exception(backoff.expo, (openai.error.RateLimitError,
                                     openai.error.APIError,
                                     openai.error.ServiceUnavailableError,),
                      max_tries=5, max_value = 60)
def augment_temp(number):
    augmented = []
    
    for n in tqdm(range(number)):
        random.shuffle(keywords)
        #그냥 생성하면 앞에 있는 키워드가 비정상적으로 많이 사용됨으로 한번 generate 할 때마다 keyword shuffle
        keywords_joined = ','.join(keywords)

        output = get_answer_from_gpt4_test(keywords_joined + "\n 위 리스트에 있는 단어들을 같이 2개 이상 랜덤으로 사용하여 사용자가 " + accommodation_type + "데이터베이스에 " + accommodation_type +"을 검색하는 중 할 만한 질문을 하나 생성해 주세요. 무조건 물음표로 질문이 끝나게 만들어 주세요.")
        #prompt에 information 더 추가하면 자세히 나옴 (예: 료칸을 검색 할 때 --> 일본여행을 계획하는 관광객이..?)
        output_data = [output]
        augmented.append(output_data)
    return augmented

get_answer_from_gpt4_test() 안에 prompt에서 "... 질문 n개 생성해줘" 안쓰는 이유:

위 prompt처럼 하나씩 생성하면 output quality가 더 좋음

In [ ]:
#csv로 저장code (append)
def save_to_csv(data, filename):
    with open(filename, 'a', newline = '', encoding='utf8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)

for i in range(반복 수):

save_to_csv(augment_temp(질문 생성하고 싶은 수), 'save할 csv파일이름')

range(반복 수) X augment_temp(숫자) = 총 생성되는 수

나누는 이유는 RateLimitError 때문

In [ ]:
for i in range(4):
    try:
        save_to_csv(augment_temp(500), 'gpt_test_ryokan123.csv')
        print("waiting for next")
        time.sleep(60) #RateLimitError 대비, 필요 시 시간 늘리기
        print("resuming")
    except:
        continue